In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Optimal Behaviour and User Modelling

Optimal user behaviour reduces a target's energy most rapidly. This involves exactly tracking the target's position, instantly absorbing all potential energy as it is converted from kinetic energy. In practice real users have constraint precision and act with delays, both present in their visual perception, cognitive process, and motor execution. To capture delays and constraint precision, we model user input as a first-order lag with Gaussian noise.

#### First Order Lag User

A first order lag system receives a repeated step function control signal.

In [ ]:

# control signal u
num_repetitions = 3
num_steps_per_phase = 10
u = np.hstack([np.zeros((num_repetitions, num_steps_per_phase)), np.ones((num_repetitions, num_steps_per_phase))]).reshape(-1)
plt.plot(u, label='u')
plt.title('control signal');
plt.xlabel('timestep t (int)');

# lag response
k = 0.5 # lag constant [0, 1]
y = np.zeros_like(u)
y[0] = 0
for t in range(1, u.shape[0]):
  y[t] = (1-k) * y[t-1] + k * u[t]
  
plt.plot(y, label='y')
plt.legend()

In [ ]:
# 2. Wrap behavior in an agent class.
class FirstOrderLag:
  """"
  First order lag (vectorized).
  
  Args:
      conductivity (np.array): moving average weight of new input, in [0,1].
      s0 (np.array): initial state after reset. 
  """
  
  def __init__(self, conductivity=0.5, s0=0):
    self.k = conductivity
    self.s0 = s0
    
  def reset(self):
    self.s = np.copy(self.s0)
    return self.s
  
  def step(self, u):
    self.s = (1-self.k) * self.s + self.k * u
    return self.s

In [ ]:
# 3. simulate agent response to control signal.
user = FirstOrderLag(conductivity=0.15)
y = [user.reset()] + [user.step(u[t]) for t in range(1, u.shape[0])]
plt.plot(u)
plt.plot(y)

## User Interface

To illustrate how active inference can help adapt parameters of complex user interfaces to improve performance, we explore a target selection task with artificially complicated but well understood interaction dynamics. A visual display shows $N$ targets in motion mimicking the harmonic oscillation of masses on springs. All springs are *anchored* at the same height to a horizontal line that is controlled by the user. Each target has a different parametrization corresponding to different oscillation frequencies. A target is selected by reducing its mechanical energy to fall below some threshold $\tau$ while keeping all other targets' mechanical energies above the threshold. The mechanical energy is defined as the sum of potential energy and kinetic energy, with all energy being kinetic at the zero crossing and all energy being potential at the oscillation extrema. The oscillation amplitude thereby gives the user feedback about the effect of their inputs on progress towards selecting individual targets. Moving the anchor towards or away from a target reduces or increase its potential energy, respectively.

The harmonic oscillators act as a common mediating mechanism between various input spaces (e.g., trajectories of a mouse pointer, a visually tracked hand, gaze, or device tilting movements) and the control space (one of $N$ target selection). By decoupling the interaction metaphor from the specific input modality we can leverage users' mental model of the dynamics to make control across multiple modalities more intuitive, provided that parametrisations of the dynamics can be found that work well for each modality and user. Traditionally, finding good parametrisations was done manually or through trial and error user testing in each new context. More recently, computational interaction methods have been proposed, where user models trained via reinforcement learning are used to evaluate individual interface parametrisations in simulation  off-line. Here, we set out to demonstrate how active inference can be used to simultaneously learn the user model and adapt interface parametrisations to optimize task performance on-line, i.e. while a user interacts with the system.

### Harmonic Oscillators

We parametrise each harmonic oscillator with a mass $m$ and a spring constant $\kappa$, which correspond to the oscillation frequency in Eq. X
$$f=\frac{1}{2\pi}\sqrt{\frac{\kappa}{m}}$$

The state is represented by $(x, \dot x)$ , where $x$ is its distance from the anchor and $\dot x$ its velocity. The mechanical energy $E$, potential energy $U$, and kinetic energy $K$ are defined as in Eq. X
$$
E = U + K 
$$
$$ U = \frac{1}{2}\kappa \cdot x^2 $$
$$ K = \frac{1}{2}m \cdot \dot x^2 $$

The dynamics are described by an ordinary differential equation (ODE) and state updates are performed by taking one Euler step of length $\delta t$ (see Eq. X)

$$ \ddot x = \frac{F}{m} = \frac{-\kappa \cdot \dot x_t}{m} $$
$$ \dot x_t = \dot x_{t-1} + \delta t \cdot \ddot x_t $$
$$ x_t = x_{t-1} + \delta t \cdot \ \dot x_t$$



In [ ]:
class HarmonicOscillator():
    
    def __init__(self, 
                 N=1, 
                 mass=1., 
                 spring_constant=1., 
                 damping_coefficient=0., 
                 dt=0.1, 
                 energy_start=1., 
                 energy_max=2):
      self.N = N
      self.mass = mass
      self.spring = spring_constant
      self.damping = damping_coefficient
      self.dt = dt # Euler step length
      self.energy_start = energy_start
      self.energy_max = energy_max
      
      self.x = None
      self.v = None
      self.anchor = 0
      
    def frequency(self):
      return 1/(2*np.pi) * np.sqrt(self.spring / self.mass)
    
    def potential_energy(self):
      return 0.5 * self.spring * (self.x - self.anchor)**2
    
    def kinetic_energy(self):
      return 0.5 * self.mass * self.v**2
    
    def energy(self):
      return self.potential_energy() + self.kinetic_energy()
    
    def ddot_x(self, dx):
      f_spring = -self.spring * dx
      f_friction = -self.damping * self.v
      f_total = f_spring + f_friction
      a = f_total / self.mass
      return a
    
    def reset(self):
      # initialise all targets with equal energy and different phase
      phase = np.linspace(0, 2*np.pi, self.N+1)[:-1]
      e = self.energy_start # total mechanical energy
      max_x = np.sqrt(2*e/self.spring) # amplitude along x
      self.x = max_x * np.sin(phase)
      # note: maximum avoids numerical instability near zero in sqrt
      k = np.maximum(0, e - self.potential_energy())
      self.v = np.sqrt(2*k/self.mass) * np.sign(np.cos(phase))
      return {'x': self.x, 'v': self.v}
        
    def step(self, action=0):
      self.anchor = action
      dx = self.x - self.anchor
      a = self.ddot_x(dx)
      self.v = self.v + self.dt * a
      x = self.x + self.dt * self.v
      self.x = x

      # boundary condition: limit energy
      e = self.energy()
      factor = np.minimum(1, self.energy_max / e)
      #print('e', e)
      #print('damping', factor)
      self.spring = self.spring * factor
      self.mass = self.mass * factor
      return {'x': self.x, 'v': self.v}
    
    def plot_phase_space(self, axis=None):
      if axis is None:
        _, axis = plt.subplots()
      num_steps = int(np.max(1/(self.frequency() * self.dt))) + 2
      outputs = ['x', 'v']
      if self.x is None:
        self.reset()
      
      states = {o: [] for o in outputs}
      for _ in range(num_steps):
        state = self.step()
        for o in outputs:
          states[o].append(state[o])
      
      plt.sca(axis)
      plt.plot(np.array(states['x']), np.array(states['v']), '-')
      plt.title('Phase space');
      plt.xlabel('$x$')
      plt.ylabel("$\dot x$")
        

In [ ]:
# 5. simulate uncontrolled MassSpringDamper
num_steps = 200
agent = HarmonicOscillator(N=1, dt=0.1, energy_start=1)
agent.plot_phase_space()

y = [agent.reset()['x']]
e = [agent.energy()]
for _ in range(num_steps):
  y.append(agent.step()['x'])
  e.append(agent.energy())

fig, axes = plt.subplots(2, 1, figsize=(2*6, 12))
plt.sca(axes[0])
plt.plot(y)
plt.sca(axes[1])
plt.plot(e)
plt.ylim([0, np.max(e)]);


In [ ]:
# simulate dynamics of oscillators with different parameters and different energy
num_objects = 5
mass = np.hstack([ 1 * np.ones((num_objects)), 3 * np.ones(num_objects)]).reshape(-1)
spring = np.hstack([ 3 * np.ones((num_objects)), 1 * np.ones(num_objects)]).reshape(-1)
agent = HarmonicOscillator(N=2*num_objects, mass=mass, spring_constant=spring)

fig, axes = plt.subplots(1, 2, figsize=(2*6, 6))
agent.plot_phase_space(axes[0])

y = [agent.reset()['x']]
for _ in range(num_steps):
  y.append(agent.step()['x'])
  
y = np.array(y)
plt.sca(axes[-1])
plt.plot(y);
plt.xlabel('t')
plt.ylabel('x')

# User-UI Interaction

## Single object to control

In [ ]:
# 6. simulate MassSpringDamper controlled by FirstOrderLag
num_steps = 200

ui = HarmonicOscillator()
# generate an uncontrolled sequence
ui.reset()
y = [ui.reset()['x']] + [ui.step()['x'] for _ in range(num_steps)]

# close the loop with a first-order lag user
user = FirstOrderLag(conductivity=0.2)
user.reset()
os_user = []
os_ui = [ui.reset()['x']]
for i in range(num_steps):
  # user response to UI stimulus
  o_user = user.step(os_ui[-1])
  os_user.append(o_user)
  
  # UI update in response to user
  os_ui.append(ui.step(o_user)['x'])
  
plt.plot(y, label='uncontrolled system')
plt.plot(os_user, label='control signal')
plt.plot(os_ui, label='controlled system')
plt.xlabel('timestep t')
plt.legend()

## Selective control

In [ ]:
# 7. simulate a set of MassSpringDampers with different mass and selective control with FirstOrderLag
num_targets = 8
num_steps = 200
user_target = 1

ui = HarmonicOscillator(N=num_targets)
# generate an uncontrolled sequence
ui.reset()
y = [ui.reset()['x']] + [ui.step()['x'] for _ in range(num_steps)]

# close the loop with a first-order lag user
user = FirstOrderLag(conductivity=0.2)
user.reset()
os_user = []
os_ui = [ui.reset()['x']]
es = [ui.energy()]
for i in range(num_steps):
  # user response to UI stimulus
  o_user = user.step(os_ui[-1][user_target])
  os_user.append(o_user)
  
  # UI update in response to user
  if i < 100:
    os_ui.append(ui.step(o_user)['x'])
  else:
    os_ui.append(ui.step(os_user[100])['x'])
    
  es.append(ui.energy())
  
fig, ax = plt.subplots(2, 1, figsize=(2*6, 12))

plt.title('Position over time')
plt.sca(ax[0])
plt.plot(os_ui)
plt.plot(np.array(y)[:,user_target], 'r--', label='uncontrolled system')
plt.plot(os_user, label='control signal')
plt.plot(np.array(os_ui)[:,user_target], 'r', label='controlled system')
plt.xlabel('timestep t')
plt.legend()

plt.sca(ax[1])
plt.title('Energy over time')
plt.plot(es)
plt.plot(np.array(es)[:,user_target], 'r')

## Selection Trigger (naive)

For a fixed level of reliability, selection speed decreases with increasing number of objects.

For a fixed threshold, reliability decreases with increased number of objects.

For a fixed number of objects, reliability increases with lower threshold.


In [ ]:
class SimpleMovingAverage:
  
  def __init__(self, buffer_size=8, s0=0):
    self.buffer_size = buffer_size
    self.s0 = s0
    
  def reset(self):
    self.buffer = np.ones((self.buffer_size,) + np.array(self.s0).shape) * self.s0
    return self.buffer.mean(axis=0)
  
  def step(self, action):
    self.buffer[:-1] = self.buffer[1:]
    self.buffer[-1] = action
    return self.buffer.mean(axis=0)
  
sma = SimpleMovingAverage(buffer_size=8)
ys = [sma.reset()] + [sma.step(u[t]) for t in range(u.shape[0])]
plt.plot(ys)
    

In [ ]:
# 8. reason about user intent by observing interaction

# a. traditional approach: moving average velocity below threshold, with
# initial value, capcity (moving average weight) and threshold finetuned
# tie-break on value if threshold is crossed by multiple targets
# - fine-tune initial value to maximum peak velocity
# - fine-tune conductivity to reduce oscillation in controlled object's score
#   near convergence
# - fine-tune threshold to separate controlled from uncontrolled objects
# - trigger action if a single object's score is below threshold

num_objects = 16
num_steps = 200
fig, ax = plt.subplots(figsize=(8*2, 12))

mass = np.ones(num_objects) + np.linspace(0, 1, num_objects)
ui = MassSpringDamper(mass=mass, bounds=[-1, 1])
user = FirstOrderLag(conductivity=0.1)

reasoner = FirstOrderLag(conductivity=0.05, s0=0.1)
#reasoner = SimpleMovingAverage(buffer_size=64, s0=np.ones(num_objects)*0.02)
threshold = 0.001

os_ui = []
os_user = []
ss_reasoner = []
os_reasoner = []

user.reset()
reasoner.reset()
o_ui = ui.reset()

for _ in range(num_steps):
  # stimulus response - target system at index 0
  o_user = user.step(o_ui['x'][-1])
  # log stimulus and response
  os_ui.append(o_ui['x'])
  os_user.append(o_user)
  # simulate next stimulus
  o_ui = ui.step(o_user)
  # reason about ui response to user input
  s_reasoner = reasoner.step(np.abs(o_ui['dx']))
  o_reasoner = s_reasoner < threshold
  ss_reasoner.append(s_reasoner)
  os_reasoner.append(o_reasoner)
  """
  if np.sum(o_reasoner > 1e-6):
    # object selected
    break
  """
  
plt.plot(ss_reasoner, '-');
plt.plot(np.max(np.array(os_reasoner), axis=1) * np.max(ss_reasoner), 'k-', label='triggered');
plt.plot([0, num_steps], [threshold, threshold], 'k--', label='threshold');
plt.xlabel('timestep t')
plt.ylabel('selection score (lower is better)')
plt.legend()

## Selection Trigger (Active Inference)

In the simplest setting, we can perform belief updates using the model of the user and of the UI dynamics to reason about the user's target object.

In a more advanced setting, we can intervene on the UI to help resolve ambiguity by changing some of the objects' parameters (mass, spring constant, and damping constant).